In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('train.csv')
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


Lets perform initial EDA

In [3]:
df.shape

(404290, 6)

In [4]:
df.sample(10)

,id,qid1,qid2,question1,question2,is_duplicate
130005,130005,208724,208725,Extra courses for ECE students?,Is it dangerous to replace your sleep schedule...,0
380381,380381,327984,511987,Where can I know about the top hotels in Bhopal?,How would I know about the top hotels in Bhopal?,0
65240,65240,113252,113253,What should I wear under my skirt?,What should I wear on long skirt?,0
163282,163282,150323,148436,Why should I be studying?,Why should I study?,0
367331,367331,88344,41083,How do I get an internship in Google?,How can I get an internship at Google?,1
92334,92334,154585,73804,How are eccentric and concentric contractions ...,What are the differences between eccentric and...,0
251953,251953,366167,366168,"Is 6'3"" (1.90m) tall for a guy?",Will a girl fall for a guy who is 6 ft tall an...,0
200729,200729,302574,302575,What is ordinary RC moment resisting frame?,How moments are created in framed structure?,0
164371,164371,255424,255425,What is something most Cloud storage Apps lack?,Cloud Storage: Do other Dropbox-like tools hav...,0
181121,181121,39332,18194,Is timetravel possible?,Is time travel theoretically possible?,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404289 non-null  object
 4   question2     404288 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [6]:
#missing values
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64

In [7]:
df.duplicated().sum()

0

In [8]:
#distribution of duplicates and non duplicates

print(df['is_duplicate'].value_counts())

is_duplicate
0    255027
1    149263
Name: count, dtype: int64


In [1]:
print(df['is_duplicate'].value_counts())
print((df['is_duplicate'].value_counts()/df['is_duplicate'].count())*100)
df['is_duplicate'].value_counts().plot(kind='bar')

NameError: name 'df' is not defined

In [10]:
# Repeated questions

qid = pd.Series(df['qid1'].tolist() + df['qid2'].tolist())
print('NUmber of unique questions',np.unique(qid).shape[0])
x = qid.value_counts()>1
print('Number of questions getting repeated',x[x].shape[0])

NUmber of unique questions 537933
Number of questions getting repeated 111780


# Lets built a model without doing annything to see performance 
Without Feature engineering

In [11]:
new_df = df.sample(30000)

In [12]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [13]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
54206,Avg antivirus 1800</v\>251<’-‘>4919 Avg tech s...,Can you suggest to me about any AVG Antivirus ...
172701,What are the best books on algorithms for a be...,What are the best algorithm books for beginners?
352925,What are some of the best Marathi movies?,"Where can I watch the Marathi movie, Sinhasan,..."
66610,Is it possible to have civilization without laws?,Will mankind survive without laws?
246317,"Who will win the election, Donald Trump or Hil...",Will Donald Trump defeat his opponent in the g...


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
#merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=3000)
q1_arr,q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [16]:
temp_df1 = pd.DataFrame(q1_arr,index=ques_df.index)
temp_df2 = pd.DataFrame(q2_arr,index=ques_df.index)
temp_df = pd.concat([temp_df1,temp_df2],axis=1)
temp_df.shape

(30000, 6000)

In [17]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [18]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
54206,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
172701,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
352925,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
66610,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
246317,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size = 0.2,random_state=2)

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred  = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7458333333333333

In [26]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.725

# Feature Engineering